# 사전을 이용한 감성분석

In [2]:
import pandas as pd

In [3]:
# 데이터 로딩
def read_data(filename, encoding='cp949'):                # 읽기 함수 정의
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]                 # txt 파일의 헤더(id document label)는 제외하기
  return data

def write_data(data, filename, encoding='cp949'):         # 쓰기 함수도 정의
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

#data = read_data('/content/gdrive/My Drive/pytest/ratings_small.txt', encoding='cp949')
data = pd.read_csv('C:/Users/jye12/Python/file/pytest_대용량/ratings(네이버영화평).txt', sep='\t', encoding='cp949')         # 전체파일은 ratings.txt (긍정 1만, 부정 1만)

In [4]:
print(len(data))
data.describe()

200000


,id,label
count,2.000000e+05,200000.000000
mean,6.739774e+06,0.500000
std,2.923504e+06,0.500001
min,3.300000e+01,0.000000
25%,4.760678e+06,0.000000
50%,7.527726e+06,0.500000
75%,9.248413e+06,1.000000
max,1.027815e+07,1.000000


In [7]:
import rhinoMorph
rn = rhinoMorph.startRhino()
roM = rhinoMorph.onlyMorph_list

filepath:  C:\Users\jye12\anaconda\lib\site-packages
classpath:  C:\Users\jye12\anaconda\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


In [8]:

# 전체 데이터 형태소 분석
#df2 = pd.DataFrame(columns = ['id','context','label'])
morphed_data = ''
for data_each in data.values:
    morphed_data_each = roM(rn,data_each[1],pos = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi=True)
    joined_data_each = ' '.join(morphed_data_each) 
    if joined_data_each:
        morphed_data += str(data_each[0]) +"\t" + joined_data_each + '\t' + str(data_each[2]) + '\n'


TypeError: No matching overloads found for rhino.RHINO.ExternCall_forPython(float,bool), options are:
	public static java.lang.String rhino.RHINO.ExternCall_forPython(java.lang.String,boolean)



In [69]:
with open('C:/Users/jye12/Python/file/pytest_대용량/ratings_morphed.txt','w', encoding= 'cp949') as f:
    for i in morphed_data.split('\n'):
        f.write(i +'\n')

In [9]:
# 형태소 분석된 데이터 로딩
data = read_data('C:/Users/jye12/Python/file/pytest_대용량/ratings_morphed.txt', encoding='cp949')

In [10]:
print(len(data)) 
print(len(data[0])) 

# print(data[0])

495
3


In [72]:
del data[-1]

In [73]:
# 감정사전 읽기
data_id = [line[0] for line in data]       # 데이터 id
data_text = [line[1] for line in data]     # 데이터 본문
data_senti = [line[2] for line in data]    # 데이터 긍부정 부분

positive = read_data('C:/Users/jye12/Python/file/pytest/positive.txt')      # 긍정 감정사전 읽기
negative = read_data('C:/Users/jye12/Python/file/pytest/negative.txt')      # 부정 감정사전 읽기
                     
pos_found = []                                # 각 문장에서 발견될 긍정어의 개수 
neg_found = []                                # 각 문장에서 발견될 부정어의 개수 

In [74]:
# 감정단어 파악

def cntWordInLine(data, senti):
    senti_found = []
    for onedata in data:
        oneline_word = onedata.split(' ')       # 한 줄의 데이터를 공백 단위로 분리하여 리스트로 저장
        senti_temp = 0
        for sentiword in senti:      
            if sentiword[0] in oneline_word:      # posword[0] 하여 리스트를 문자열로 추출
                senti_temp += 1                     # 현재의 감정단어와 일치하면 숫자를 하나 올려 줌 (중복은 계산 안 함)
        senti_found.append(senti_temp)          # 현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
    return senti_found
      
      
data_senti_poscnt = cntWordInLine(data_text, positive)      # 발견된 긍정 단어의 숫자 파악
data_senti_negcnt = cntWordInLine(data_text, negative)      # 발견된 부정 단어의 숫자 파악

print(data_senti_poscnt)
print(data_senti_negcnt)

[2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 3, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 3, 4, 0, 1, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [75]:
# 감정점수 계산
# Pandas 데이터프레임으로 저장
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text, 'original':data_senti, 
                        'pos':data_senti_poscnt, 'neg':data_senti_negcnt})
senti_score = newdata['pos'] - newdata['neg']      # 긍정개수에서 부정개수를 뺌
newdata['senti_score'] = senti_score               # 그 수를 senti_score 컬럼에 저장 

newdata.loc[newdata.senti_score > 0, 'new'] = 1    # 새로운 긍부정 기호
newdata.loc[newdata.senti_score <= 0, 'new'] = 0   # 새로운 긍부정 기호

# 처음에 기록된 긍부정과 새로 계산된 긍부정이 같은지 여부를 matched 컬럼에 저장
# original 컬럼은 문자로 되어 있으므로 숫자로 변환 뒤 비교 
newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True'
newdata.loc[pd.to_numeric(newdata.original) != newdata.new, 'matched'] = 'False'

In [76]:
# 원점수와 비교 및 저장
score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() + newdata.matched.str.count('False').sum()) * 100
print(score)

newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False) # csv 저장
newdata.to_csv('newfile2.txt', sep='\t', encoding='cp949', index=False) # 또는 txt 저장

59.716599190283404


In [77]:
# 시그모이드 점수 계산
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)

In [78]:
# 결과 확인
pd.set_option('display.max_columns', 10)     # 보여지는 컬럼 수 조정
print(newdata.iloc[:5, :])

         id                                               text original  pos  \
0   8132799  디자인 배우다 학생 외국 디자이너 일구다 전통 통하다 발전 문화 산업 부럽다 사실 ...        1    2   
1   4655635                                 폴리스스토리 시리즈 뉴 없다 최고        1    1   
2   9251303                  와 연기 진짜 쩔다 지루 생각하다 몰입 그래 이런 진짜 영화        1    0   
3  10067386                          안개 자욱 하다 밤하늘 뜨다 초승달 같다 영화        1    0   
4   2190435                            사랑 해보다 사람 처음 끝 웃다 있다 영화        1    1   

   neg  senti_score  new matched   sigmoid  
0    0            2  1.0    True  0.880797  
1    0            1  1.0    True  0.731059  
2    1           -1  0.0   False  0.268941  
3    0            0  0.0   False  0.500000  
4    0            1  1.0    True  0.731059  



# 머신러닝을 이용한 감성분석 

In [9]:
# 형태분석된 데이터 로딩
import pandas as pd

def write_data(data, filename, encoding='cp949'):        # 쓰기 함수
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

data = pd.read_csv('./origin/ratings_morphed.txt',sep ='\t', encoding='cp949')

print(len(data))
print(len(data.values[0])) 
print(data.values[0])

197514
3
[8132799
 '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사'
 1]


In [12]:
# 훈련 데이터와 테스트데이터 분리 (싸이킷런 모듈)
data_text = [line[1] for line in data.values]
data_senti = [line[2] for line in data.values]
from sklearn.model_selection import train_test_split
train_data_text , test_data_text, train_data_senti , test_data_senti = train_test_split(data_text , data_senti, stratify=data_senti)

# Counter 를 이용해 잘 분류되었는지 확인하기
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq :' , train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)

print('test_data_senti_freq : ' , test_data_senti_freq)

train_data_senti_freq : Counter({1: 74097, 0: 74038})
test_data_senti_freq :  Counter({1: 24699, 0: 24680})


In [45]:
# 훈련 데이터 테스트데이터 분리 (직접)
import random
random.shuffle(data.values)

data_70 = int(len(data) *0.7)
train_data = data[:data_70]
test_data = data[data_70:]

print('train data length : ', len(train_data))
print('test data length : ' , len(test_data))


# 학습데이터 요소 분리
train_data_text = [line[1] for line in train_data.values]      # 훈련데이터 본문
train_data_senti = [line[2] for line in train_data.values]     # 훈련데이터 긍부정 부분



# 테스트데이터 요소 분리
test_data_text = [line[1] for line in test_data.values]

test_data_senti = [line[2] for line in test_data.values]

train data length :  138259
test data length :  59255


In [46]:
# 행렬 형태로 변환
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_data_text)  # 문서 빈도 5이상의 단어만 대상
X_train = vect.transform(train_data_text)          # 행렬 생성
print("X_train:\n", repr(X_train))         #생성된 행렬 개요

X_train:
 <138259x11058 sparse matrix of type '<class 'numpy.int64'>'
	with 763966 stored elements in Compressed Sparse Row format>


In [47]:
#행렬 내용 관찰
feature_names = vect.get_feature_names()

print('특성 개수 : ' , len(feature_names))
print('처음 20개 특성 : \n', feature_names[:20])
print('3000~5000까지의 특성 : \n', feature_names[3000:5000])

특성 개수 :  11058
처음 20개 특성 : 
 ['10점', '1빠', 'cgv', 'ebs', 'jtbc', 'kbs', 'la', 'mb', 'mbc', 'naver', 'new', 'ok', 'sbs', 'sns', 'tv', 'ㄴㄴ', 'ㄷㄷ', 'ㅂㄷㅂㄷ', 'ㅂㅅ', 'ㅅㅂ']
3000~5000까지의 특성 : 
 ['만능', '만담', '만델라', '만두', '만들', '만들수', '만듦새', '만땅', '만만', '만만세', '만명', '만물', '만발', '만보', '만사', '만세', '만약', '만인', '만점', '만족', '만족감', '만지', '만찬', '만하', '만해', '만행', '만화', '만화가', '만화영화', '만화책', '만회', '많이', '말기', '말길', '말년', '말로', '말론', '말리', '말미', '말빨', '말살', '말세', '말씀', '말아먹', '말없이', '말입', '말자', '말장난', '말종', '말초신경', '말코비치', '말콤', '말투', '말하', '말하기', '말해', '맘껏', '맘대로', '맘마', '맘마미아', '맘속', '맙소사', '맛깔', '맛나', '맛보', '맛보기', '맛없', '맛있', '맛집', '망가', '망가지', '망각', '망상', '망설이', '망설임', '망신', '망인', '망쳐놓', '망치', '망하', '맞대', '맞먹', '맞물리', '맞서', '맞이', '맞추', '맞춤', '맡기', '매개체', '매국노', '매기', '매끄럽', '매너', '매년', '매니아', '매니저', '매달', '매달리', '매도', '매드', '매들린', '매듭', '매력', '매력적', '매료', '매미', '매번', '매수', '매염방', '매우', '매일', '매일매일', '매장', '매주', '매직', '매진', '매체', '매치', '매튜', '매트', '매트릭스', '매혹', '매혹적', '매회', '맥가이버', '맥과이어', '맥그리거', '맥라이언

In [48]:
# 머신러닝 알고리즘의 적용
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

y_train = pd.Series(train_data_senti)
print(len(y_train))
lr = LogisticRegression(solver="liblinear")

138259


In [49]:
lr.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [50]:
# 테스트 데이터 입력

X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
print('테스트 데이터 점수 :' , lr.score(X_test, y_test))

테스트 데이터 점수 : 0.6020926504092482


In [51]:
# 그리드 서치를 통한 최적 매개변수 찾기

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.01, 0.1, 1, 3, 5]}          # 적절한 C 값의 범위를 설정한다. C는 규제 -> 과대적합,과소적합을 조정가능
grid = GridSearchCV(LogisticRegression(solver="liblinear"), param_grid, cv=5)
grid.fit(X_train, y_train)

print("최고 교차 검증 점수:", round(grid.best_score_, 3))
print("최적의 매개변수:", grid.best_params_)

X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)

print("테스트 데이터 점수:", grid.score(X_test, y_test))


최고 교차 검증 점수: 0.84
최적의 매개변수: {'C': 1}
테스트 데이터 점수: 0.6020926504092482


# 1개 데이터 예측 

In [52]:
import rhinoMorph as rM
rn = rM.startRhino()

# 형태소 분석
new_input = '오늘은 정말 재미있는 하루구나!'
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(rn, new_input , pos =['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input) 

inputdata.append(morphed_input)
print('input_data : ', inputdata)

filepath:  C:\Users\jye12\anaconda\lib\site-packages
classpath:  C:\Users\jye12\anaconda\lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
input_data :  ['오늘 정말 재미있 하루']


In [53]:
X_input = vect.transform(inputdata)
result = lr.predict(X_input)

if result== '0':
    print('부정적인 글')
else:
    print('긍정적인 글')

긍정적인 글


C:\Users\jye12\anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.
